In [1]:
import uuid
import time
import pickle
import sys
import gym.spaces
import itertools
import numpy as np
import random
import tensorflow                as tf
import tensorflow.contrib.layers as layers
from collections import namedtuple
import TicTacToe
import Players


/Users/christophermiller/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()

def TicTacToe_model(placeholder, num_actions, scope, reuse=tf.AUTO_REUSE):
    # A model for a TicTacToe q-function
    placeholder = tf.contrib.layers.flatten(placeholder)
    with tf.variable_scope(scope, reuse=reuse):
        out = placeholder
        out = tf.cast(out, tf.float32)
        out = tf.layers.dense(out, 64  , bias_initializer = tf.zeros_initializer(), activation = tf.nn.softmax)
        out = tf.layers.dense(out, 64  , bias_initializer = tf.zeros_initializer(), activation = tf.nn.softmax)
        out = tf.layers.dense(out, 9 , kernel_initializer = tf.zeros_initializer(), bias_initializer = tf.zeros_initializer(), activation = None)
    return out

    
# def update_old_model_initialize_new_model(old_model, new_model, board_placeholder, sess, update_iter):
#     old_model = new_model
#     old_model_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="model" +str(update_iter-1))
#     
#     new_model = TicTacToe_model(board_placeholder, 9, scope = "model" + str(update_iter))
#     
#     new_model_vars =  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="model" +str(update_iter))
#     sess.run(tf.variables_initializer(new_model_vars))
#     
#     for var_old, var_new in zip(old_model_vars, new_model_vars):
#         var_new = sess.run(var_old)
# #        print(sess.run(var_old))
# #        print(sess.run(var_new))
# #    print(sess.run(old_model, feed_dict = {board_placeholder: [[[1,0,0],[0,0,0],[0,0,0]]]})[0,:3])
# #    print(sess.run(new_model, feed_dict = {board_placeholder: [[[1,0,0],[0,0,0],[0,0,0]]]})[0,:3])
#     return old_model, new_model
# 
# def update_and_duplicate_old_model():
#     old_model = new_model
#     
#     variables = tf.trainable_variables()
#     for var1 in variables:
#         if "pred/" in var1.name:
#             trained_var = [var2 for var2 in tf.global_variables() if var2.op.name in str.replace(var1.name, "pred/", "")][0]
#             value = sess.run(trained_var)
#             sess.run(tf.assign(var1, value))


def sample_action(model, mask_placeholder):
    out = model
    dist = tf.distributions.Categorical(probs=maskedSoftmax(out, mask_placeholder))
    return dist.sample()
    
    
"""Code from online"""
def maskedSoftmax(logits, mask):
    """
    Masked softmax over dim 1
    :param logits: (N, L)
    :param mask: (N, L)
    :return: probabilities (N, L)
    """
    indices = tf.where(mask)
    values = tf.gather_nd(logits, indices)
    denseShape = tf.cast(tf.shape(logits), tf.int64)
    
    """THIS IS THE KEY: tensorflow will automatically set output probabilities to zero of undesignated entries in sparse vector"""
    sparseResult = tf.sparse_softmax(tf.SparseTensor(indices, values, denseShape))
    
    result = tf.scatter_nd(sparseResult.indices, sparseResult.values, sparseResult.dense_shape)
    result.set_shape(logits.shape)
    return result

# def sample_trajectory_random_opponent(new_model, sess, env):
#     new_sample = sample_action(new_model)
#     obs, acs, rewards = [], [], []
#     seed = np.random.randint(0,2)
#     ob = env.reset()
#     if seed % 2 == 0:
#         while True:
#             obs.append(ob)
#             action = sess.run(new_sample, feed_dict={board_placeholder: [ob]})
#             acs.append(action)
#             ob, rew, done, _ = env.step(action[0])
#             rewards.append(rew)
#             
#             if done:
#                 break
#             
#             action = [env.action_space.sample()]
#             ob, rew, done, _ = env.step(action[0])
#     else:
#         while True:       
#             action = [env.action_space.sample()]
#             ob, rew, done, _ = env.step(action[0])
#    
#             obs.append(ob)
#             action = sess.run(new_sample, feed_dict={board_placeholder: [ob]})
#             acs.append(action)
#             ob, rew, done, _ = env.step(action[0])
#             rewards.append(rew)
#             
#             if done:
#                 break
#                 
#     path = {"observation" : np.array(obs, dtype=np.float32), 
#                 "reward" : np.array(rewards, dtype=np.float32), 
#                 "action" : np.array(acs, dtype=np.float32)}
# #    print(str(len(path["observation"])))
#     return path
    

def sample_trajectory(player, opponent, env):
    """player:   realization of Player.Player abstract class
       opponent: realization of Player.Player abstract class
       env:      environment which follows open ai gym environment structure and has a current_player int either 1 or 2
    
       realizes a rollout of env using player and opponent policy
       
       ouputs a path dictionary with keys: observation, reward, action
       Each yields a 1D np array which shows the observation, reward, action pair at every point of the rollout
    """
    
    obs, acs, rewards, masks = [], [], [], []
    ob = env.reset()
    done = False
    player_has_acted = False
    action = None
    
    #Do rest of moves
    while not done:
        #Get current observation of current player
        ob = env.get_observation(env.current_player)
        legal_moves = env.legal_moves()
        if env.current_player == 1:
            #Reward is recorded as results of state,action pair... need to check player 1 has acted already
            if player_has_acted:
                rewards.append(env.get_reward(1))
            else:
                player_has_acted = True
                
            action = player.policy(np.array([ob]), np.array([legal_moves]))
            obs.append(ob)
            acs.append(action[0])
            masks.append(legal_moves)
        else:
            action = opponent.policy(np.array([ob]), np.array([legal_moves]))
        done, _ = env.step(action[0]) 

    #Need to record final reward for player 1
    rewards.append(env.get_reward(1))
    
    path = {"observation" : np.array(obs, dtype=np.int32), 
                "reward" : np.array(rewards, dtype=np.float32), 
                "action" : np.array(acs, dtype=np.int32),
                "mask" : np.array(masks, dtype=np.int32)}
    return path

    
    
    

# def sample_trajectories(player, opponent, env, min_timesteps_per_batch):
#     paths =[]
#     timesteps_this_batch = 0
#     while True:
#         path = sample_trajectory(player, opponent, env)
#         paths.append(path)
#         timesteps_this_batch += len(path['observation'])
#         if timesteps_this_batch > min_timesteps_per_batch:
#             break
#     return paths
    
def sum_of_rewards(paths, gamma = 1): 
    re_n = [path["reward"] for path in paths]
    q_n = []
    for seq_of_rewards in re_n:
        for t in range(len(seq_of_rewards)):
            weighted_sequence = seq_of_rewards[t:] * np.array([gamma**i for i in range(len(seq_of_rewards[t:]))])
            q_n.append(np.sum(weighted_sequence))
    adv_n = q_n
    return adv_n
        
def standardize_advantage(adv_n):
    adv_n = (adv_n - np.mean(adv_n)) 
    adv_n = adv_n * (1.0/(np.std(adv_n)+.0000001))
    return adv_n

def get_log_prob(model, action_placeholder, mask_placeholder):
    action_dim = 9 
    logits = model
    
    indices = tf.where(mask_placeholder)
    values = tf.gather_nd(logits, indices)
    denseShape = tf.cast(tf.shape(logits), tf.int64)
    
    """THIS IS THE KEY: tensorflow will automatically set output probabilities to zero of undesignated entries in sparse vector"""
    sparseResult = tf.sparse_softmax(tf.SparseTensor(indices, values, denseShape))
    
    probability_dist = tf.scatter_nd(sparseResult.indices, sparseResult.values, sparseResult.dense_shape)
#     probability_dist = probability_dist.set_shape(logits.shape)
    
    """Want to emulate this:"""
#     probability_dist = tf.nn.softmax(logits)
#     legal_pseudo_probability_dist = probability_dist*values
#     legalprobability_dist = tf.divide(legal_pseudo_probability_dist, tf.reduce_sum(legal_pseudo_probability_dist, axis= 1))
    
    prod = tf.multiply(probability_dist, tf.one_hot(action_placeholder, action_dim ))
    
    log_prob = tf.log(tf.reduce_sum(prod , axis = 1 ))
#    log_prob = -tf.nn.sparse_softmax_cross_entropy_with_logits(labels= action_placeholder, logits= tf.SparseTensor(indices, values, denseShape))
    return log_prob

def loss_and_update_op(log_prob,adv_n):
    loss = -tf.reduce_mean(log_prob * adv_n) 
    optimizer = tf.train.AdamOptimizer(5e-3)
    update_op = optimizer.minimize(loss)
    return loss, update_op, optimizer
    

In [10]:
tf.reset_default_graph()

#define the board, models *symbolically*
observation_placeholder = tf.placeholder(shape = [None, 3,3], dtype = tf.int32)
adv_n_placeholder = tf.placeholder(shape = [None], dtype = tf.float32)
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32)
mask_placeholder = tf.placeholder(shape=[None, 9], dtype = tf.int32)


model = TicTacToe_model(observation_placeholder, 9, scope = "model-1", reuse=tf.AUTO_REUSE)
#old_model = TicTacToe_model(board_placeholder, 9, scope = "model-2", reuse=tf.AUTO_REUSE)
model_input_s = sample_action(model, mask_placeholder)

#Define Loss functions *symbolically*
log_prob = get_log_prob(model, action_placeholder, mask_placeholder)
loss, update_op, optimizer = loss_and_update_op(log_prob, adv_n_placeholder)

#start a session
sess =tf.Session()
sess.run(tf.global_variables_initializer())
#Defines player, opponent
player = Players.NN_Player(model, model_input_s, sess, observation_placeholder, mask_placeholder, duplicate=False)
opponent = Players.Random_Player()

#Loads old player,opponent
temp_file_name = './bot_10_28_v6.ckpt'

#Want to duplicate session
saver = tf.train.Saver()
saver.restore(sess, temp_file_name)


opponent = Players.NN_Player(model, model_input_s, sess, observation_placeholder, mask_placeholder)




#start an environment
env = TicTacToe.TicTacToe()

number_updates_per_expert_update = 200
number_expert_updates = 1000

for k in range(number_expert_updates):
    print("iteration number", k)

    #Initialize all variables first time only, otherwise update only uptimizer vars
#     if k ==0:
#         sess.run(tf.global_variables_initializer())
#     else:
#         sess.run(tf.variables_initializer(optimizer.variables()))
    
    batch_adv_n = []
    batch_wins = []
    
    tic = time.time()
    for i in range(number_updates_per_expert_update):
        #Produce some trajectories
#        print("sampling games...")
        max_time_step = 100
        time_step = 0
        paths = []
        while time_step < max_time_step:
#             print("sampling game...")
            path = sample_trajectory(player,opponent,env)
#             print("game sampled.")
            paths.append(path)
            time_step += len(path['reward'])
#             print(path['observation'])
            batch_wins.append(env.current_winner)
#         paths = sample_trajectories(player, opponent, env, 100)

        adv_n = sum_of_rewards(paths)
        batch_adv_n = batch_adv_n + adv_n
        

        boards = np.concatenate([path['observation'] for path in paths])
        masks = np.concatenate([path['mask'] for path in paths])
        actions = np.squeeze(np.concatenate([path["action"] for path in paths])).astype(int)
        
        sess.run(update_op, feed_dict = {mask_placeholder: masks, adv_n_placeholder: adv_n, observation_placeholder: boards , action_placeholder: actions})

    
    #Unwind win data:
    batch_win_unique, batch_win_counts = np.unique(batch_wins, return_counts=True)
    batch_win_count, batch_draw_count, batch_loss_count = 0,0,0
    try:
        batch_win_count = dict(zip(batch_win_unique, batch_win_counts))[1]*1.0
        batch_draw_count = dict(zip(batch_win_unique, batch_win_counts))[0]*1.0
        batch_loss_count = dict(zip(batch_win_unique, batch_win_counts))[2]*1.0
    except KeyError as e:
        print("error")
        
    print(dict(zip(batch_win_unique, batch_win_counts)))
    print("mean adv", np.mean(batch_adv_n))
    print("win:loss ratio", batch_win_count/(batch_loss_count + 1))
    print("iteration time", time.time() - tic)
    print(path)
    
    if batch_win_count/(batch_loss_count + 1)  > 4:
        opponent = Players.NN_Player(model, model_input_s, sess, observation_placeholder,mask_placeholder)
            
    


Destroying NN_Player and Session...
Destroying NN_Player and Session...
INFO:tensorflow:Restoring parameters from ./bot_10_28_v6.ckpt
duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
iteration number 0
{0: 2, 1: 4252, 2: 3222}
mean adv 0.30953206239168113
win:loss ratio 1.3192677629537697
iteration time 25.20523977279663
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 1, 0],
        [0, 0, 2]],

       [[2, 0, 1],
        [0, 1, 0],
        [0, 0, 2]]], dtype=int32), 'reward': array([0., 0., 1.], dtype=float32), 'action': array([4, 2, 6], dtype=int32), 'mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 1, 0]], dtype=int32)}
iteration number 1
{0: 7, 1: 6089, 2: 505}
mean adv 0.8484773458776925
win:loss ratio 12.033596837944664
iteration time 23.37377405166626
{'observation': array([[[0, 0, 0],
        [0

Destroying NN_Player and Session...
iteration number 12
{0: 120, 1: 3273, 2: 2973}
mean adv 0.147077713551702
win:loss ratio 1.1005379959650303
iteration time 26.994688034057617
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 2],
        [0, 1, 0],
        [0, 0, 0]],

       [[0, 0, 2],
        [0, 1, 2],
        [0, 0, 1]],

       [[0, 0, 2],
        [1, 1, 2],
        [2, 0, 1]],

       [[2, 1, 2],
        [1, 1, 2],
        [2, 0, 1]]], dtype=int32), 'reward': array([0., 0., 0., 0., 1.], dtype=float32), 'action': array([4, 8, 3, 1, 7], dtype=int32), 'mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=int32)}
iteration number 13
{0: 111, 1: 4002, 2: 2143}
mean adv 0.3250975935168256
win:loss ratio 1.8666044776119404
iteration time 25.76027512550354
{'observation': array([[[0, 0, 0],
        [0, 2, 0

{0: 872, 1: 2514, 2: 1191}
mean adv 0.3788169346511285
win:loss ratio 2.109060402684564
iteration time 25.09615731239319
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 2],
        [0, 1, 0],
        [0, 0, 0]],

       [[0, 0, 2],
        [2, 1, 1],
        [0, 0, 0]],

       [[1, 0, 2],
        [2, 1, 1],
        [0, 0, 2]],

       [[1, 1, 2],
        [2, 1, 1],
        [2, 0, 2]]], dtype=int32), 'reward': array([0., 0., 0., 0., 1.], dtype=float32), 'action': array([4, 5, 0, 1, 7], dtype=int32), 'mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=int32)}
iteration number 25
{0: 2116, 1: 2257, 2: 168}
mean adv 0.521895119673662
win:loss ratio 13.355029585798816
iteration time 25.170878887176514
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 2],
        [0, 1,

{0: 4485, 1: 32, 2: 15}
mean adv 0.003933523453633592
win:loss ratio 2.0
iteration time 25.546982049942017
{'observation': array([[[0, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 2, 2],
        [0, 0, 0]],

       [[2, 0, 1],
        [1, 2, 2],
        [0, 0, 0]],

       [[2, 2, 1],
        [1, 2, 2],
        [0, 0, 1]]], dtype=int32), 'reward': array([0., 0., 0., 0.], dtype=float32), 'action': array([2, 3, 8, 7], dtype=int32), 'mask': array([[1, 1, 1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 0]], dtype=int32)}
iteration number 36
{0: 4482, 1: 32, 2: 14}
mean adv 0.00398034398034398
win:loss ratio 2.1333333333333333
iteration time 25.229525566101074
{'observation': array([[[0, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 2, 2],
        [0, 0, 0]],

       [[2, 0, 1],
        [1, 2, 2],
        [0, 0, 0]],

       [[2, 2, 1],
        [1, 2

{0: 4509, 1: 13, 2: 4}
mean adv 0.0017712177121771218
win:loss ratio 2.6
iteration time 25.273494005203247
{'observation': array([[[0, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 2, 2],
        [0, 0, 0]],

       [[2, 0, 1],
        [1, 2, 2],
        [0, 0, 0]],

       [[2, 0, 1],
        [1, 2, 2],
        [0, 2, 1]]], dtype=int32), 'reward': array([0., 0., 0., 0.], dtype=float32), 'action': array([2, 3, 8, 1], dtype=int32), 'mask': array([[1, 1, 1, 1, 0, 1, 1, 1, 1],
       [1, 1, 0, 1, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 1, 0, 0]], dtype=int32)}
iteration number 48
{0: 4507, 1: 7, 2: 4}
mean adv 0.0004917146088410287
win:loss ratio 1.4
iteration time 25.177869081497192
{'observation': array([[[0, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[0, 0, 1],
        [0, 2, 2],
        [0, 0, 0]],

       [[2, 0, 1],
        [1, 2, 2],
        [0, 0, 0]],

       [[2, 2, 1],
        [1, 2, 2],
        [

...

error
{0: 4515, 1: 1}
mean adv 0.00024547105896214835
win:loss ratio 1.0
iteration time 24.203750133514404
{'observation': array([[[2, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[2, 0, 2],
        [0, 1, 0],
        [0, 0, 0]],

       [[2, 1, 2],
        [0, 1, 0],
        [0, 2, 0]],

       [[2, 1, 2],
        [2, 1, 1],
        [0, 2, 0]]], dtype=int32), 'reward': array([0., 0., 0., 0.], dtype=float32), 'action': array([4, 1, 5, 6], dtype=int32), 'mask': array([[0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int32)}
iteration number 946
error
{0: 4515, 1: 1}
mean adv 0.00019648295510364477
win:loss ratio 1.0
iteration time 24.173556804656982
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[1, 2, 1],
        [0, 2, 0],
        [0, 0, 0]],

       [[1, 2, 1],
        [0, 2, 2],
       

error
{0: 4535}
mean adv 0.0
win:loss ratio 0.0
iteration time 29.649330854415894
{'observation': array([[[2, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[2, 0, 2],
        [0, 1, 0],
        [0, 0, 0]],

       [[2, 1, 2],
        [0, 1, 0],
        [0, 2, 0]],

       [[2, 1, 2],
        [2, 1, 1],
        [0, 2, 0]]], dtype=int32), 'reward': array([0., 0., 0., 0.], dtype=float32), 'action': array([4, 1, 5, 6], dtype=int32), 'mask': array([[0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int32)}
iteration number 958
error
{0: 4527, 1: 1}
mean adv 0.00014726817534730743
win:loss ratio 1.0
iteration time 28.050765991210938
{'observation': array([[[2, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[2, 0, 2],
        [0, 1, 0],
        [0, 0, 0]],

       [[2, 1, 2],
        [0, 1, 0],
        [0, 2, 0]],

       [[2, 1, 2],
        [2, 1, 1],
        [0, 2, 0]]], dtype=int32

error
{0: 4514, 1: 2}
mean adv 0.00044278264292039755
win:loss ratio 2.0
iteration time 24.5262930393219
{'observation': array([[[2, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[2, 0, 2],
        [0, 1, 0],
        [0, 0, 0]],

       [[2, 1, 2],
        [0, 1, 0],
        [0, 2, 0]],

       [[2, 1, 2],
        [2, 1, 1],
        [0, 2, 0]]], dtype=int32), 'reward': array([0., 0., 0., 0.], dtype=float32), 'action': array([4, 1, 5, 6], dtype=int32), 'mask': array([[0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int32)}
iteration number 970
error
{0: 4521, 2: 1}
mean adv -0.0001968503937007874
win:loss ratio 0.0
iteration time 25.71336007118225
{'observation': array([[[2, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[2, 0, 2],
        [0, 1, 0],
        [0, 0, 0]],

       [[2, 1, 2],
        [0, 1, 0],
        [0, 2, 0]],

       [[2, 1, 2],
        [2, 1, 1],
        [0

error
{0: 4525, 1: 1}
mean adv 0.00024568817257137243
win:loss ratio 1.0
iteration time 25.50905132293701
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[1, 2, 1],
        [0, 2, 0],
        [0, 0, 0]],

       [[1, 2, 1],
        [0, 2, 2],
        [0, 1, 0]],

       [[1, 2, 1],
        [1, 2, 2],
        [2, 1, 0]]], dtype=int32), 'reward': array([0., 0., 0., 0., 0.], dtype=float32), 'action': array([0, 2, 7, 3, 8], dtype=int32), 'mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)}
iteration number 982
error
{0: 4520, 1: 3}
mean adv 0.0005896226415094339
win:loss ratio 3.0
iteration time 24.565295219421387
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

    

error
{0: 4524, 2: 2}
mean adv -0.0003925417075564279
win:loss ratio 0.0
iteration time 24.88205075263977
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[1, 2, 1],
        [0, 2, 0],
        [0, 0, 0]],

       [[1, 2, 1],
        [0, 2, 2],
        [0, 1, 0]],

       [[1, 2, 1],
        [1, 2, 2],
        [2, 1, 0]]], dtype=int32), 'reward': array([0., 0., 0., 0., 0.], dtype=float32), 'action': array([0, 2, 7, 3, 8], dtype=int32), 'mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)}
iteration number 994
error
{0: 4528}
mean adv 0.0
win:loss ratio 0.0
iteration time 24.95987820625305
{'observation': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[1, 0, 0],
        [0, 2, 0],
        [0, 0, 0]],

       [[1, 2, 1],
        [0

In [4]:
temp_file_name = './bot_10_28_v7.ckpt'

#Want to duplicate session
saver = tf.train.Saver()
saver.save(sess, temp_file_name)

'./bot_10_28_v6.ckpt'

In [9]:
temp_file_name = './bot_10_28_v3.ckpt'

#Want to duplicate session
saver = tf.train.Saver()
saver.restore(sess, temp_file_name)

INFO:tensorflow:Restoring parameters from ./bot_10_28_v3.ckpt


In [ ]:
human = Players.Human_Player()
sample_trajectory(human,opponent,env)


1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Say something: 2
[2]
1
[[0. 0. 1.]
 [0. 2. 0.]
 [0. 0. 0.]]
Say something: 7
[7]
1
[[2. 0. 1.]
 [0. 2. 0.]
 [0. 1. 0.]]
Say something: 8
[8]
1
[[2. 2. 1.]
 [0. 2. 0.]
 [0. 1. 1.]]
Say something: 3
[3]
1
[[2. 2. 1.]
 [1. 2. 2.]
 [0. 1. 1.]]


In [19]:
tf.reset_default_graph()

#define the board, models
board_placeholder = tf.placeholder(shape = [None, 3,3], dtype = tf.int32)
adv_n_placeholder = tf.placeholder(shape = [None], dtype = tf.float32)
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32)
new_model = TicTacToe_model(board_placeholder, 9, scope = "model-1", reuse=tf.AUTO_REUSE)

#start a session
sess =tf.Session()

#start an environment
env = TicTacToe.TicTacToe()

number_updates_per_expert_update = 10
number_expert_updates = 4

#Define Loss functions *symbolically*
log_prob = get_log_prob(new_model, action_placeholder, mask_placeholder)
loss, update_op, optimizer = loss_and_update_op(log_prob, adv_n_placeholder)

#Initialize all variables first time only, otherwise update only uptimizer vars
sess.run(tf.global_variables_initializer())

for i in range(20):
        #Produce some trajectories
        paths = sample_trajectories_random_opponent(new_model, sess, 20, env = env)
        adv_n = sum_of_rewards(paths) 
        boards = np.concatenate([path['observation'] for path in paths])
        actions = np.squeeze(np.concatenate([path["action"] for path in paths])).astype(int)
        test_vars =  tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="model" +str(-1))
        for var in test_vars:
            print(var.name, np.max(sess.run(var)))
        sess.run(update_op, feed_dict = {adv_n_placeholder: adv_n, board_placeholder: boards , action_placeholder: actions})


TypeError: get_log_prob() missing 1 required positional argument: 'mask_placeholder'

In [18]:

def sample_trajectory(player, opponent, env):
    """player:   realization of Player.Player abstract class
       opponent: realization of Player.Player abstract class
       env:      environment which follows open ai gym environment structure and has a current_player int either 1 or 2
    
       realizes a rollout of env using player and opponent policy
       
       ouputs a path dictionary with keys: observation, reward, action
       Each yields a 1D np array which shows the observation, reward, action pair at every point of the rollout
    """
    
    obs, acs, rewards = [], [], []
    ob = env.reset()
    while True:
        if env.current_player == 1:
            obs.append(ob)
            player_action = player.policy(np.array([ob]))
            acs.append(player_action[0])
            ob, rew, done, _ = env.step(player_action[0])
            rewards.append(rew)
            
            if done:
                break
        else:
            opponent_action = opponent.policy(np.array([ob]), env.legal_moves())
            ob, rew, done, _ = env.step(opponent_action[0])
                
    path = {"observation" : np.array(obs, dtype=np.float32), 
                "reward" : np.array(rewards, dtype=np.float32), 
                "action" : np.array(acs, dtype=np.float32)}
    return path
